In [28]:
import ast
import fitz
import io
import pandas as pd
from tqdm import tqdm
import requests

In [19]:
filtered_df = pd.read_csv("/home/dominik/Documents/Informatik/1_Semester/medLLM/data/pmc_patients/PMC-Patients-oa-9995.csv")
article_ids = filtered_df["PMID"].tolist()
linked_ids_scores = filtered_df["relevant_articles"].tolist()

articles_PMID = set()
articles_PMID.update(article_ids)

In [20]:
for id_score in linked_ids_scores:
    data_dict = ast.literal_eval(id_score)
    for article_id in data_dict.keys():
        articles_PMID.add(article_id)


print(len(articles_PMID))

153252


In [21]:
articles = pd.DataFrame(list(articles_PMID), columns=["PMID"])
articles.head()

,PMID
0,29310370
1,24680818
2,24812003
3,34832013
4,17968826


In [22]:
def is_open_access(pmid):
    url = f"https://www.ebi.ac.uk/europepmc/webservices/rest/search?query=EXT_ID:{pmid}&resultType=core&format=json"
    response = requests.get(url)
    
    if response.status_code == 200:
        data = response.json()
        if "resultList" in data and data["resultList"]["result"]:
            result = data["resultList"]["result"][0]
            if result.get("isOpenAccess", "N") == "Y":
                return data["resultList"]["result"][0]["fullTextUrlList"]["fullTextUrl"][0]["url"]
        return None

In [ ]:
type(articles[""])

pandas.core.series.Series

In [ ]:
tqdm.pandas()


articles['pdf_url'] = articles['PMID'].head(1000).progress_apply(is_open_access)
print(articles)

In [27]:
first_1000 = articles.head(1000)

In [29]:
def extract_text_from_pdf(url):
    if url is None:
        return None
    try:
        response = requests.get(url)
        if response.status_code == 200:
            pdf_file = io.BytesIO(response.content)
            pdf_document = fitz.open(stream=pdf_file, filetype="pdf")
            text = ""
            for page in pdf_document:
                text += page.get_text()
            pdf_document.close()
            return text
        else:
            return f"Failed to download PDF (status code: {response.status_code})"
    except Exception as e:
        return str(e)

In [ ]:
tqdm.pandas(desc="Extracting text from PDFs")

first_1000['pdf_text'] = first_1000['pdf_url'].progress_apply(extract_text_from_pdf)

first_1000.head(100)

In [ ]:
first_1000.to_csv("/home/dominik/Documents/Informatik/1_Semester/medLLM/data/pmc_patients/1000_articles.csv")

In [ ]:
articles_pdfs = first_1000[first_1000["pdf_url"].notna()]
failed_downloads = articles_pdfs[articles_pdfs["pdf_text"].str.contains("Failed to download PDF|Failed to open stream", na=False)]
num_failed_downloads = failed_downloads.shape[0]

print(f"Number of rows containing 'Failed to download PDF' or 'Failed to open stream': {num_failed_downloads}")

1000 articles; 331 available, 182 PDF to text failed because of access/streamfailures etc, 149 pdf to text conversions successful